In [1]:
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import fsolve, brentq

## **Datos Iniciales de la Instalación**

In [2]:
# @markdown Elevaciones en metros:
elevacion_succion  = 0 # @param {"type":"number"}
elevacion_descarga = 80 # @param {"type":"number"}
# @markdown Longitud de tubería en metros:
longitud_tuberia = 750 # @param {"type":"number"}
# @markdown Diámetro de tubería en mm:
diametro = 160 # @param {"type":"number"}
C_rugosidad = 120 # @param {"type":"number"}




## **Datos del equipo de bombeo (ficha técnica)**

In [24]:
# Datos del fabricante: se deben tener la misma cantidad de datos
Qn = np.array([0, 10, 20, 30, 40, 50, 60, 70, 75])  # Caudales (L/s)
Hn = np.array([132, 133, 132, 131, 128, 122, 117, 98, 88])  # Alturas de bombeo  (mca)

# Dato opcional: curva de la eficiencia o rendimiento del equipo de bombeo
#nef = np.array([])  # Eficiencia sin datos
nef = np.array([10, 20, 52, 63, 70, 73.5, 73.5, 69, 64.5])  # Eficiencia (%)

## **Funciones de cálculo**

### *Curva característica*
#### $ Hb = A*Qb^2 + B*Qb + C $

### *Curva de eficiencia*
#### $ nef = E*Qb^2 + D*Qb + F $

### *Curva resistente de la instalación*
#### $ Hr = (z2-z1) + R*Q^2 $

In [22]:
def calcular_curva_resistente(Q, Z1, Z2, L, D, C):
    """
    Calcula la curva resistente de la tubería.
    """
    R = 10.674 / C**1.852 / ((D / 1000)**4.871) * L
    return (Z2 - Z1) + R * (Q / 1000)**1.852

def curva_resistente(Q):
    return calcular_curva_resistente(Q, elevacion_succion, elevacion_descarga, longitud_tuberia, diametro, C_rugosidad)

def ajuste_polinomial(Qn, valores, grado):
    """
    Ajusta un polinomio de grado especificado a los datos (Qn, Hn) o (Qn, nef).
    """
    if len(valores)==0:
        return None, None
    else:
      coeficientes = np.polyfit(Qn, valores, grado)
      polinomio = np.poly1d(coeficientes)
      return polinomio, coeficientes

def evaluar_ajuste(Qn, Hn, polinomio):
    """
    Calcula errores y coeficientes de determinación para validar el ajuste.
    """
    fi = polinomio(Qn)
    errores = fi - Hn
    Hm = np.mean(Hn)
    sr = np.sum((Hn - fi)**2)
    st = np.sum((Hn - Hm)**2)
    r2 = (st - sr) / st
    return errores, r2

def calcular_punto_operacion(curva_bomba, curva_resistente, caudal_inicial, caudal_final, diametro):
    """
    Encuentra el caudal de operación resolviendo la intersección de las curvas.
    """
    def diferencia(Q):
        return curva_bomba(Q) - curva_resistente(Q)

    # Verificar si las curvas tienen signos opuestos en los extremos del intervalo
    signo_inicial = diferencia(caudal_inicial)
    signo_final = diferencia(caudal_final)

    if signo_inicial * signo_final > 0:
        # Si no hay cambio de signo, imprimir mensaje y detener ejecución
        print("La curva resistente no se intersecta con la curva característica. Punto de operación fuera de la curva de la bomba. Revisa tus parámetros.")
        return None, None, None

    Q_operacion = brentq(diferencia, caudal_inicial, caudal_final)
    H_operacion = curva_bomba(Q_operacion)

    #calculo velocidad del flujo
    velocidad = (Q_operacion/1000) / (np.pi * (diametro / 1000)**2 / 4)

    return Q_operacion, H_operacion, velocidad

def graficar_curvas(curva_bomba, curva_resistente, Qn, Hn, punto_operacion, curva_eficiencia=None):

    Qi = np.linspace(0, max(Qn)*1.05, 100)
    Hb = curva_bomba(Qi)

    if punto_operacion[0] is not None:
      Qi_res = np.linspace(0, punto_operacion[0]*1.2, 100)
    else:
      Qi_res = np.linspace(0, max(Qn)*1.05, 100)

    Hr = curva_resistente(Qi_res)

    fig = go.Figure()

    # Curva de la bomba
    fig.add_trace(go.Scatter(x=Qi, y=Hb, mode='lines', name='Curva de la bomba', hovertemplate='Q: %{x:.2f} L/s<br>H: %{y:.2f} m'))

    # Curva resistente
    fig.add_trace(go.Scatter(x=Qi_res, y=Hr, mode='lines', name='Curva resistente', hovertemplate='Q: %{x:.2f} L/s<br>H: %{y:.2f} m'))

    # Datos de la bomba
    fig.add_trace(go.Scatter(x=Qn, y=Hn, mode='markers', name='Datos bomba', marker=dict(size=8, color='skyblue'), hovertemplate='Q: %{x:.2f} L/s<br>H: %{y:.2f} m'))

    # Punto de operación
    if punto_operacion[0] is not None:
      fig.add_trace(go.Scatter(x=[punto_operacion[0]], y=[punto_operacion[1]], mode='markers+text',
                              name='Punto de operación',
                              marker=dict(size=10, color='red'),
                              hovertemplate='Q: %{x:.2f} L/s<br>H: %{y:.2f} m'
                              ))
      fig.update_layout(yaxis_title="Altura (m)")

    if curva_eficiencia:
        nef_ajustada = curva_eficiencia(Qi)
        fig.add_trace(go.Scatter(x=Qi, y=nef_ajustada, mode='lines', name='Curva de eficiencia (%)', line=dict(dash='dash', color='gray'), hovertemplate='Q: %{x:.2f} L/s<br>nef: %{y:.2f} %'))
        fig.add_trace(go.Scatter(x=Qn, y=nef, mode='markers', name='Datos eficiencia (%)', marker=dict(size=8, color='gainsboro')))

        # Punto de operación curva eficiencia
        if punto_operacion[0] is not None:
          eficiencia_op = curva_eficiencia(punto_operacion[0])
          fig.add_trace(go.Scatter(x=[punto_operacion[0]], y=[eficiencia_op], mode='markers+text',
                                  name='Punto de operación',
                                  marker=dict(size=10, color='red'),
                                  hovertemplate='Q: %{x:.2f} L/s<br>nef: %{y:.2f} %'
                                  ))
          fig.update_layout(yaxis_title="Altura (m) / Eficiencia (%)")

    fig.update_layout(
        title="CURVAS DE OPERACIÓN DE LA BOMBA",
        xaxis_title="Caudal (L/s)",
        showlegend=False,
        template="plotly_white",
        grid=dict(rows=1, columns=1)
    )
    fig.show()

    if punto_operacion[2] is not None:
      print(f'El punto de operación será: Qb ={punto_operacion[0]:.2f} L/s y Hb ={punto_operacion[1]:.2f} mca')
      print(f'La velocidad del flujo será de {punto_operacion[2]:.2f} m/s')

## **Cálculos y resultados**

In [25]:
# --- AJUSTES ---
# Ajuste para la curva de la bomba
curva_bomba, coef_bomba = ajuste_polinomial(Qn, Hn, grado=2)

# Ajuste para la curva de eficiencia
curva_eficiencia, coef_eficiencia = ajuste_polinomial(Qn, nef,  grado=2)

# Ecuación de Curva resistente
curva_resistente = lambda Q: calcular_curva_resistente(Q, elevacion_succion, elevacion_descarga, longitud_tuberia, diametro, C_rugosidad)

# --- CÁLCULO DEL PUNTO DE OPERACIÓN ---
Q_operacion, H_operacion, velocidad = calcular_punto_operacion(curva_bomba, curva_resistente, 0, max(Qn), diametro)

# --- GRÁFICO ---
punto_operacion = (Q_operacion, H_operacion, velocidad)
graficar_curvas(curva_bomba, curva_resistente, Qn, Hn, punto_operacion, curva_eficiencia=curva_eficiencia)

El punto de operación será: Qb =53.69 L/s y Hb =117.78 mca
La velocidad del flujo será de 2.67 m/s
